In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import ShuffleSplit

In [73]:
#creating whole dataset
seventeen = pd.read_json("https://missingmigrants.iom.int/global-figures/2017/json")
sixteen = pd.read_json("https://missingmigrants.iom.int/global-figures/2016/json")
fifteen = pd.read_json("https://missingmigrants.iom.int/global-figures/2015/json")
fourteen = pd.read_json("https://missingmigrants.iom.int/global-figures/2014/json")

frames = [seventeen,sixteen,fifteen,fourteen]



In [101]:
df = pd.concat(frames)
#cleaning data
df = df.replace('', 0,regex=True)
df = df.replace('nan', 0, regex=True)
df.drop(['Information Source'])
df = df[df['Web ID'] != 42424] #drops row with -1 as number of survivors


df['Reported Date'] = pd.to_datetime(df['Reported Date'])
df['total people'] = pd.to_numeric(df['Number of survivors']) + pd.to_numeric(df['Total Dead and Missing'])
df['percent female'] = (pd.to_numeric(df['Number of Female'])/ df['total people']).fillna(0)
df['percent male'] =  (pd.to_numeric(df['Number of Male'])/ df['total people']).fillna(0)
df['percent kids'] =  (pd.to_numeric(df['Number of Children']) / df['total people']).fillna(0)
df['month'] = df['Reported Date'].dt.month
df['day'] = df['Reported Date'].dt.day

# the index will help us get back from number to category later
##df['UNSD Geographical Grouping'], geoIndex = pd.Series(df['UNSD Geographical Grouping']).factorize()
#df['Migrant Route'], migrantIndex = pd.Series(df['Migrant Route']).factorize()
#df['Region of Incident'], regionIndex = pd.Series(df['Region of Incident']).factorize()
# one hot coding
ohc_UNSD = pd.get_dummies(df['UNSD Geographical Grouping'],prefix='UNSD')
ohc_Route = pd.get_dummies(df['Migrant Route'],prefix='Route')
ohc_Region = pd.get_dummies(df['Region of Incident'],prefix='Region')

df = df.join(ohc_UNSD)
df = df.join(ohc_Region)
df = df.join(ohc_Route)
df

,Cause of death,Information Reliability,Information Source,Location,Location Description,Migrant Route,Number Dead,Number Missing,Number of Children,Number of Female,...,Route_Dominican Republic to Puerto Rico,Route_Eastern Mediterranean,"Route_From Haiti to Dajabon, Dominican Republic",Route_Horn Africa to Yemen,Route_Italy to France,Route_Malaysia to Indonesia,Route_Venezuela to Curaçao,Route_Western African,Route_Western Balkans,Route_Western Mediterranean
0,Presumed drowning,Verified,0,"35.882833113479, -5.803099736328",Unspecified location off the coast of Morocco,Western Mediterranean,0,3,0,0,...,0,0,0,0,0,0,0,0,0,1
0,Presumed drowning,Verified,0,"35.882833113479, -5.803099736328",Unspecified location off the coast of Morocco,Western Mediterranean,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
0,Presumed drowning,Verified,0,"35.882833113479, -5.803099736328",Unspecified location off the coast of Morocco,Western Mediterranean,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
0,Presumed drowning,Verified,0,"35.882833113479, -5.803099736328",Unspecified location off the coast of Morocco,Western Mediterranean,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
0,Presumed drowning,Verified,0,"35.882833113479, -5.803099736328",Unspecified location off the coast of Morocco,Western Mediterranean,0,3,0,0,...,0,0,0,0,0,0,0,0,0,1
0,Presumed drowning,Verified,0,"35.882833113479, -5.803099736328",Unspecified location off the coast of Morocco,Western Mediterranean,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
0,Presumed drowning,Verified,0,"35.882833113479, -5.803099736328",Unspecified location off the coast of Morocco,Western Mediterranean,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
0,Presumed drowning,Verified,0,"35.882833113479, -5.803099736328",Unspecified location off the coast of Morocco,Western Mediterranean,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
0,Presumed drowning,Verified,0,"35.882833113479, -5.803099736328",Unspecified location off the coast of Morocco,Western Mediterranean,0,3,0,0,...,0,0,0,0,0,0,0,0,0,1
0,Presumed drowning,Verified,0,"35.882833113479, -5.803099736328",Unspecified location off the coast of Morocco,Western Mediterranean,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
#separating data
fourteen = df[(df['Reported Date'] >= '2014-01-01') & (df['Reported Date'] < '2015-01-01')]
fifteen = df[(df['Reported Date'] >= '2015-01-01') & (df['Reported Date'] < '2016-01-01')]
sixteen = df[(df['Reported Date'] >= '2016-01-01') & (df['Reported Date'] < '2017-01-01')]
seventeen = df[(df['Reported Date'] >= '2017-01-01') & (df['Reported Date'] < '2018-01-01')]

In [96]:
total_data = np.concatenate((np.array(fourteen),np.array(fifteen),np.array(sixteen),np.array(seventeen)))
np.random.shuffle(total_data)
y = total_data[:,11].astype(int) / (total_data[:,14].astype(int) + total_data[:,11].astype(int)) 
# y is what we predict -- total sruvivors over total survivors+missing and dead 
y = np.nan_to_num(y)
y = (100*y).astype(int)
# x is the parts of the data we are using to predict Location, route, region, and Geographical Grouping
x[1]

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([0, 'US-Mexico Border', 'Northern America'], dtype=object)

In [97]:
train = 2000
x_tr = x[:train,:]
y_tr = y[:train]
x_ts = x[train:,:]
y_ts = y[train:]
from sklearn import linear_model

In [137]:
def initTest(df):
    y = np.array((pd.to_numeric(df['Number of survivors']) / df['total people']).fillna(0))
    y = np.nan_to_num(y)
    y[y == np.abs(np.inf)] = 0
    
    y = (100*y).astype(int)


    x = np.array(df[['percent female', 'percent kids', 'percent male', 'day','month']])
    x = np.column_stack((x,np.array(df.loc[:,'UNSD_0':])))
    
    x = np.nan_to_num(x)
    x[:,0] = (100*x[:,0]).astype(int)
    x[:,1] = (100*x[:,1]).astype(int)
    x[:,2] = (100*x[:,2]).astype(int)
    
    rs = ShuffleSplit(n_splits=3, test_size=.25, random_state=0)

    for train_index, test_index in rs.split(x):
        regr = linear_model.LinearRegression()
        regr.fit(x[train_index],y[train_index])
        print("train: ", regr.score(x[train_index],y[train_index]) )
        print("test: ", regr.score(x[test_index],y[test_index]))
        print(regr.coef_)
        print()
    print()
initTest(sixteen)

train:  0.116304629477
test:  0.103027432459
[ -1.40166244e-01   2.38499945e-02  -1.32281096e-01   9.14345796e-03
  -3.47463591e-01   1.39423519e+10   1.39423519e+10   1.39423519e+10
   1.39423519e+10   1.39423520e+10   1.39423519e+10   1.39423519e+10
   1.39423519e+10   1.39423519e+10   1.39423519e+10   1.39423519e+10
   1.39423519e+10   1.39423519e+10   1.39423519e+10   1.39423519e+10
   1.39423519e+10   1.39423519e+10   1.39423519e+10  -3.07085010e+10
  -3.07085010e+10  -3.07085010e+10  -3.07085010e+10  -3.07085010e+10
  -3.07085010e+10  -3.07085010e+10  -3.07085010e+10  -3.07085010e+10
  -3.07085010e+10  -3.07085010e+10  -3.07085010e+10  -3.07085010e+10
  -3.07085010e+10   4.52929373e+10   4.52929373e+10   4.52929373e+10
   4.52929373e+10   4.52929374e+10   4.52929373e+10   4.52929373e+10
   4.52929374e+10   4.52929373e+10   4.52929374e+10   4.52929373e+10
   4.52929373e+10   4.52929374e+10   4.52929373e+10   4.52929374e+10
   4.52929373e+10]

train:  0.113125764802
test:  0.112806

In [138]:
onlyUS = df[df['UNSD Geographical Grouping'] == geoIndex.get_loc('Northern America')]

In [139]:
initTest(onlyUS)
initTest(df)
initTest(df[df['UNSD Geographical Grouping'] == geoIndex.get_loc('Central America')])

ValueError: Found array with 0 sample(s) (shape=(0, 53)) while a minimum of 1 is required.